# Benchmarking Results

In [1]:
import pandas as pd

### 1. Load, Summarize, Insert

In [2]:
data = pd.read_csv('../benchmarks/load_summarize_insert/results.csv')
data.head()

,replicate,num_abstracts,total_replicate_time,total_summarization_time,total_insert_time,total_loading_time,mean_total_time,mean_summarization_time,mean_insert_time,mean_loading_time
0,1,5,46.017187,45.617424,0.399505,0.000258,9.203437,9.123485,0.079901,0.000052
1,2,5,69.757739,69.402214,0.355265,0.000259,13.951548,13.880443,0.071053,0.000052
2,3,5,59.380586,58.939484,0.440871,0.000232,11.876117,11.787897,0.088174,0.000046
3,4,5,71.140245,70.719397,0.420602,0.000246,14.228049,14.143879,0.084120,0.000049
4,5,5,51.084247,50.722499,0.361550,0.000198,10.216849,10.144500,0.072310,0.000040


### 2. Huggingface Device Map

In [3]:
data = pd.read_csv('../benchmarks/huggingface_device_map/results.csv')
data.head()

,abstract_num,summarization_time,device_map_strategy
0,1,14.629446,CPU only
1,2,16.952201,CPU only
2,3,10.698561,CPU only
3,4,13.310327,CPU only
4,5,14.418210,CPU only


### 3. Parallel Summarization

In [4]:
data = pd.read_csv('../benchmarks/parallel_summarize/results.csv')
data.head()

,num_jobs,device_map_strategy,summarization_time
0,4,GPU,296.744658
1,8,GPU,161.350356
2,12,GPU,152.624606
3,1,CPU physical cores only,1852.936123
4,2,CPU physical cores only,1196.622907


### 4. SQL Insert

In [5]:
data = pd.read_csv('../benchmarks/sql_insert/results.csv')
data.head()

,num_abstracts,insert_time,insert_strategy,replicate
0,10000,4.258317,execute_many,1
1,10000,4.022138,execute_many,2
2,10000,4.073744,execute_many,3
3,10000,4.109033,execute_many,4
4,10000,4.187685,execute_many,5
